# Sprint RNN

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

# 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

a
t
=
x
t
⋅
W
x
+
h
t
−
1
⋅
W
h
+
B
h
t
=
t
a
n
h
(
a
t
)
a
t
 : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

h
t
 : 時刻tの状態・出力 (batch_size, n_nodes)

x
t
 : 時刻tの入力 (batch_size, n_features)

W
x
 : 入力に対する重み (n_features, n_nodes)

h
t
−
1
 : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)

W
h
 : 状態に対する重み。 (n_nodes, n_nodes)

B
 : バイアス項 (n_nodes,)

初期状態 
h
0
 は全て0とすることが多いですが、任意の値を与えることも可能です。

上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
x
 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [6]:
class XavierInitializer():
    def __init__(self, n_nodes1,n_nodes2):
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2        
    
    def W(self):
        sigma = np.sqrt(1.0 / self.n_nodes1)
        self.W = sigma * np.random.randn(self.n_nodes1, self.n_nodes2)
        return self.W
    
    def B(self):
        sigma = np.sqrt(1.0 / self.n_nodes1)
        self.B = sigma * np.random.randn(self.n_nodes2)
        return self.B

In [7]:
class Adam:
    #Adam (http://arxiv.org/abs/1412.6980v8)
    def __init__(self , lr = 1e-3 , beta1 = 0.9 , beta2 = 0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self , params , grads):
        if self.m is None:
            self.m , self.v = [] , []
            for param in params:
                self.m.append(np.zeros_like(param))
                self.v.append(np.zeros_like(param))
                
        self.iter += 1
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)
        
        for i in range(len(params)):
            self.m[i] += (1 - self.beta1) * (grads[i] - self.m[i])
            self.v[i] += (1 - self.beta2) * (grads[i]**2 - self.v[i])
            
            params[i] -= lr_t * self.m[i] / (np.sqrt(self.v[i]) + 1e-7)            

In [8]:
class AdaGrad():
    def __init__(self , lr):
        self.lr = lr
        self.hw = None
        self.hb = None
    
    def update(self , layer):
#         layer.W = self.W
#         layer.B = self.B
#         layer.dW = self.dW
#         layer.dB = self.dB
        
        layer.hw += (layer.dW) * (layer.dW)
        layer.W -= self.lr * (layer.dW) / (np.sqrt(layer.hw) + 1e-7)
        layer.hb += (layer.dB) * (layer.dB)
        layer.B -= self.lr * (layer.dB) / (np.sqrt(layer.hb) + 1e-7)
        self.hw = layer.hw
        self.hb = layer.hb
        return layer.W, layer.B  

In [9]:
class SimpleRNN:
    
    def __init__(self, n_nodes,  initializer, optimizer , sigma=0.01):
        self.optimizer = optimizer
        self.n_nodes = n_nodes
        self.initializer = initializer
        self.sigma = sigma
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.Wx = self.initializer.W(n_features , n_nodes)
        self.Wh = self.initializer.W(n_nodes , n_nodes)
        self.B = self.initializer.B(n_nodes)
        self.X = None
        self.h_next = None
        self.h_prev = None
        self.dW = None
        self.dB = None
        self.hw = 0
        self.hb = 0        
        self.cache = None
        
    def forward(self , X , h_prev=0):
        self.X = X
#         self.batch_size , selfsequences , self.features = self.X.shape
#         self.a = np.zeros((self.batch_size , selfsequences , self.n_nodes))
#         self.h = np.zeros((self.batch_size , selfsequences , self.n_nodes))
        
        for n in range(n_sequences):
            if n == 0:
                at = np.dot(self.X[: , n , :] , self.Wx) + self.B
                h_s = np.tanh(at)
                self.cache = h_s
            else:
                at = np.dot(self.X[: , n , :] , self.Wx) + np.dot(self.cache , self.Wh) + self.B
                h_s = np.tanh(at)
                self.cache = h_s
        return h_s
    
    def backward(self , dh_next):
        self.X , self.h_prev , self.h_next = self.cache
        
        dt = dh_next * (1 - self.h_next**2)
        db = np.sum(dt , axis = 0)
        dWh = np.dot(self.h_prev.T , dt)
        dh_prev = np.dot(dt , self.Wh.T)
        dWx = np.dot(X.T , dt)
        dx = np.dot(dt , self.Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

# 【問題2】小さな配列でのフォワードプロパゲーションの実験

In [10]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [11]:
class SimpleRNN():
    
    def __init__(self , n_nodes , Wx , Wh , B):
        self.n_nodes = n_nodes
        self.Wx = Wx
        self.Wh = Wh
        self.B = B
        self.cache = None
    def forward(self , X):
        self.X = X
        batch_size , n_sequences , n_features = self.X.shape
    #     h_s = np.zeros((batch_size, n_nodes))

        for n in range(n_sequences):
            if n == 0:
                at = np.dot(self.X[: , n , :] , self.Wx) + self.B
                h_s = np.tanh(at)
                self.cache = h_s
            else:
                at = np.dot(self.X[: , n , :] , self.Wx) + np.dot(self.cache , self.Wh) + self.B
                h_s = np.tanh(at)
                self.cache = h_s
        return h_s

In [12]:
rnn = SimpleRNN(n_nodes , w_x , w_h , b)

In [13]:
rnn.forward(x)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

# 【問題3】（アドバンス課題）バックプロパゲーションの実装